In [23]:
import pandas as pd
import os
import warnings

warnings.filterwarnings("error")

source_path = './data/raw/full.csv'

header_df   = pd.read_csv(source_path, nrows=1, header=None)
all_header  = header_df.head(1).iloc[0].tolist()
header = [header for header in all_header if header not in ['Source', 'Description', 'Country'] ]
header

with open(source_path, 'r', encoding='utf-8') as file:
    chunk_full = pd.read_csv(file, usecols=header, chunksize=10000)

    for chunk in chunk_full:
        try:
            chunk['Start_Time'] = chunk['Start_Time'].str.replace(r'\.\d{3,}', '', regex=True)
            chunk['Start_Time'] = pd.to_datetime(chunk['Start_Time'], format='%Y-%m-%d %H:%M:%S', errors='coerce')
    
            chunk['End_Time']   = chunk['End_Time'].str.replace(r'\.\d{3,}', '', regex=True)
            chunk['End_Time']   = pd.to_datetime(chunk['End_Time'], format='%Y-%m-%d %H:%M:%S', errors='coerce')
    
            chunk['local_year']  = chunk['Start_Time'].dt.strftime("%Y").astype(int)
        except Exception as e:
            print(count, "==========")
            print(e)
            break
        
        years = chunk['local_year'].drop_duplicates().tolist()
        if 2023 in years:
            years.remove(2023)

        for year in years:
            df = chunk[chunk['local_year'] == year]
            df = df.drop('local_year', axis=1)
    
            file_path = f'./data/processed/{year}.csv'
            df.to_csv(file_path, mode='a', header=not os.path.exists(file_path), index=False)
            break

In [32]:
full.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7728394 entries, 0 to 7728393
Data columns (total 43 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   ID                     object 
 1   Severity               int64  
 2   Start_Time             object 
 3   End_Time               object 
 4   Start_Lat              float64
 5   Start_Lng              float64
 6   End_Lat                float64
 7   End_Lng                float64
 8   Distance(mi)           float64
 9   Street                 object 
 10  City                   object 
 11  County                 object 
 12  State                  object 
 13  Zipcode                object 
 14  Timezone               object 
 15  Airport_Code           object 
 16  Weather_Timestamp      object 
 17  Temperature(F)         float64
 18  Wind_Chill(F)          float64
 19  Humidity(%)            float64
 20  Pressure(in)           float64
 21  Visibility(mi)         float64
 22  Wind_Direction    

In [4]:
full['Start_Time']    = pd.to_datetime( full['Start_Time'].str.replace(r'\.\d{3,}', '', regex=True) )
full['End_Time']      = pd.to_datetime( full['End_Time'].str.replace(r'\.\d{3,}', '', regex=True) )

In [5]:
full['local_year']         = full['Start_Time'].dt.strftime("%Y").astype(int)

In [6]:
full.drop(columns=['Source', 'Description', 'Country'])
local_year = full['local_year'].drop_duplicates().to_list()
df_by_year = {}
for year in local_year:
    df = full[full['local_year'] == year]
    df_by_year[year] = df

In [11]:
for year in df_by_year:
    print(year)
    df = df_by_year[year]
    df.to_csv(f'./data/processed/full-{year}.csv', index=False)

2016
2017
2022
2021


OSError: [Errno 28] No space left on device